In [1]:
!pip -q install transformers
!pip -q install youtube_dl
!pip -q install moviepy==2.0.0.dev2

In [2]:
import os
import torch
import librosa

import moviepy.editor as mp

from IPython.display import Audio
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC

In [3]:
# Substitute below YT link
clip = "https://www.youtube.com/watch?v=7Ood-IE7sx4"

# Substitue below for start/end seconds
start = 1
end = 60

In [4]:
os.system('youtube-dl {} --recode-video mp4'.format(clip))
os.system('mv *.mp4 clip.mp4')

256

### Load Model and Tokenizer

In [5]:
# Load Wav2Vec from huggingface
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:358: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  FutureWarning,
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Extract Audio

In [6]:
clip = mp.VideoFileClip("clip.mp4")
end = min(clip.duration, end)

# Save the paths for later
clip_paths = []

# Extract Audio-only from mp4
for i in range(start, int(end), 10):
  sub_end = min(i+10, end)
  sub_clip = clip.subclip(i,sub_end)

  sub_clip.audio.write_audiofile("audio_" + str(i) + ".mp3")
  clip_paths.append("audio_" + str(i) + ".mp3")

chunk:  45%|████▌     | 100/221 [00:00<00:00, 997.53it/s, now=None]

MoviePy - Writing audio in audio_1.mp3


MoviePy - Done.


chunk:  43%|████▎     | 95/221 [00:00<00:00, 935.28it/s, now=None]

MoviePy - Writing audio in audio_11.mp3


...

MoviePy - Writing audio in audio_41.mp3


MoviePy - Done.


chunk:  53%|█████▎    | 105/199 [00:00<00:00, 1029.89it/s, now=None]

MoviePy - Writing audio in audio_51.mp3


MoviePy - Done.


In [7]:
# Play Audio 
Audio(clip_paths[0])

### Transcribe

In [8]:
cc = ""

for path in clip_paths:
    # Load the audio with the librosa library
    input_audio, _ = librosa.load(path, 
                                sr=16000)

    # Tokenize the audio
    input_values = tokenizer(input_audio, return_tensors="pt", padding="longest").input_values

    # Feed it through Wav2Vec & choose the most probable tokens
    with torch.no_grad():
      logits = model(input_values).logits
      predicted_ids = torch.argmax(logits, dim=-1)

    # Decode & add to our caption string
    transcription = tokenizer.batch_decode(predicted_ids)[0]
    cc += transcription + " "

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

In [9]:
print(cc)

SELF I GO HINE KNOCK KNOCK I SAY KNOCK KNOCK OUT WHILE THEN THEY SAID HOW TO THEY FRSE IT THEN THEY SAID A GET US SOME MONY YOU D UM GINGER YOU LEAVE YOUR TITLE BECAUSE YOU DON'T LIKE YOUR NINETY FOUR YEAR OLD GRANDMA UM THAT'S THE DEFINITION OF SOMETHING YOU WADE OUT GANGSTA ANHE SAYS YOU HAVE A BEST MY REWARDS CARD AND I SAID NO I WISH I A YOUNG AN I SAID I'M YOUNG BUT I LIED I'M THIRTY THRIN AND HE SAID EXCUSE M I AM HOMELESS I AM GAY I HAVE ADES I'M NEWINTOWN FIVE BA AND THEY GO YOU'RE A LITTLE FAT GIRL AREN'T YOU AND I GO NO AND THEY GO SAY IT AND I GO IMA LITTLE FAT GIRL NO NALL I GOT PROMOTED O ILING TO WHAT PASS AROUND PARTY BOU A HAG I AM DAMP ALL THE TIME I AM DAMP NOW AND I W 
